In [ ]:
import json

# Load the training challenges and solutions
with open('arc-agi_training_challenges.json', 'r') as file:
    training_challenges = json.load(file)

with open('arc-agi_training_solutions.json', 'r') as file:
    training_solutions = json.load(file)

# Load the evaluation challenges and solutions
with open('arc-agi_evaluation_challenges.json', 'r') as file:
    evaluation_challenges = json.load(file)

with open('arc-agi_evaluation_solutions.json', 'r') as file:
    evaluation_solutions = json.load(file)

In [12]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import json

# Define the padding function
def pad_grid(grid, target_size=(30, 30)):
    if grid.ndim != 2:
        raise ValueError(f"Grid should have 2 dimensions, but has {grid.ndim}")
    padded_grid = np.zeros(target_size, dtype=int)
    padded_grid[:grid.shape[0], :grid.shape[1]] = grid
    return padded_grid

# Define the dataset class
class ARCDataset(Dataset):
    def __init__(self, challenges, solutions):
        self.challenges = challenges
        self.solutions = solutions
        self.keys = list(challenges.keys())
        self.valid_keys = self._filter_valid_keys()

    def __len__(self):
        return len(self.valid_keys)

    def __getitem__(self, idx):
        key = self.valid_keys[idx]
        challenge = self.challenges[key]
        solution = self.solutions[key]

        # Extracting and padding the train and test grids
        train_inputs = [pad_grid(np.array(pair['input'])) for pair in challenge['train']]
        train_outputs = [pad_grid(np.array(pair['output'])) for pair in challenge['train']]
        test_inputs = [pad_grid(np.array(pair['input'])) for pair in challenge['test']]
        test_outputs = [pad_grid(np.array(output)) for output in solution]

        # Convert to tensors
        train_inputs = torch.tensor(train_inputs, dtype=torch.float32)
        train_outputs = torch.tensor(train_outputs, dtype=torch.float32)
        test_inputs = torch.tensor(test_inputs, dtype=torch.float32)
        test_outputs = torch.tensor(test_outputs, dtype=torch.float32)

        # Ensure the inputs have the correct dimensions
        train_inputs = train_inputs.unsqueeze(1)
        train_outputs = train_outputs.unsqueeze(1)
        test_inputs = test_inputs.unsqueeze(1)
        test_outputs = test_outputs.unsqueeze(1)

        return train_inputs, train_outputs, test_inputs, test_outputs

    def _filter_valid_keys(self):
        valid_keys = []
        for key in self.keys:
            challenge = self.challenges[key]
            if all('input' in pair and np.array(pair['input']).ndim == 2 for pair in challenge['train']) and \
               all('output' in pair and np.array(pair['output']).ndim == 2 for pair in challenge['train']) and \
               all('input' in pair and np.array(pair['input']).ndim == 2 for pair in challenge['test']):
                valid_keys.append(key)
            else:
                print(f"Skipping challenge {key} due to invalid grid dimensions")
        return valid_keys

# Load data from JSON files
with open('arc-agi_training_challenges.json') as f:
    training_challenges = json.load(f)
with open('arc-agi_training_solutions.json') as f:
    training_solutions = json.load(f)
with open('arc-agi_evaluation_challenges.json') as f:
    evaluation_challenges = json.load(f)
with open('arc-agi_evaluation_solutions.json') as f:
    evaluation_solutions = json.load(f)
with open('arc-agi_test_challenges.json') as f:
    test_challenges = json.load(f)

# Create DataLoader for training data
train_dataset = ARCDataset(training_challenges, training_solutions)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Define the neural network model
class ARCNN(nn.Module):
    def __init__(self):
        super(ARCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 30 * 30, 512)
        self.fc2 = nn.Linear(512, 30 * 30)

    def forward(self, x):
        batch_size, num_pairs, _, height, width = x.size()
        x = x.view(batch_size * num_pairs, 1, height, width)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.view(batch_size, num_pairs, 1, height, width)
        return x

# Instantiate the model, define loss function and optimizer
model = ARCNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for train_inputs, train_outputs, _, _ in train_loader:
        optimizer.zero_grad()
        outputs = model(train_inputs)
        target_size = outputs.size()
        train_outputs = train_outputs.view(target_size)
        loss = criterion(outputs, train_outputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.6f}')

Epoch [1/10], Loss: 1.195794
Epoch [2/10], Loss: 1.054697
Epoch [3/10], Loss: 0.904320
Epoch [4/10], Loss: 0.779642
Epoch [5/10], Loss: 0.750139
Epoch [6/10], Loss: 0.708998
Epoch [7/10], Loss: 0.618955
Epoch [8/10], Loss: 0.544355
Epoch [9/10], Loss: 0.495360
Epoch [10/10], Loss: 0.463073


In [14]:
# Evaluation function
def evaluate_model(model, data_loader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for test_inputs, test_outputs, _, _ in data_loader:
            outputs = model(test_inputs)
            target_size = outputs.size()
            test_outputs = test_outputs.view(target_size)
            loss = criterion(outputs, test_outputs)
            total_loss += loss.item()
    return total_loss / len(data_loader)

# Create DataLoader for evaluation data
evaluation_dataset = ARCDataset(evaluation_challenges, evaluation_solutions)
evaluation_loader = DataLoader(evaluation_dataset, batch_size=1, shuffle=False)

# Evaluate the model
evaluation_loss = evaluate_model(model, evaluation_loader)
print(f'Evaluation Loss: {evaluation_loss:.6f}')

Evaluation Loss: 1.784493
